In [ ]:
import pandas as pd
import re
from typing import Union
from rich.progress import track
import requests
from geopy.geocoders import ArcGIS

In [ ]:
df = pd.read_csv('../data/Medical_Examiner_Case_Archive.csv')
print(df.shape)
df.sample(1)

In [ ]:
df2 = df[df['Incident Address'].notna()]
df2.shape

In [ ]:
df2['Incident Address'].value_counts()

In [ ]:
'unk' in 'UNKNOWN location'.lower()

In [ ]:
re.sub(r"apt.*|\#.*|.*nh,", "", "wowzer NH, cool thin # 12".lower()).strip()

In [ ]:
car_words = ('auto', 'motor')

def deal_with_commas(x: str) -> str:
    if ',' not in x:
        return x.strip().title()
    parts = x.split(',')
    result = ' '.join([z for z in parts if any(y for y in z if y.isnumeric())])
    return result.strip().title()


def remove_apartment_info(x: str) -> str:
    result = re.sub(r"apt.*|\#.*|.*nh,", "", x)
    return deal_with_commas(result)


def clean_address(row: pd.Series) -> Union[int, str]:
    cause = row.get('Primary Cause Line A')
    a = row.get('Incident Address')
    # removes if motor vehicle cause
    if pd.notna(cause):
        if any(word in cause.lower() for word in car_words):
            return pd.NA
    # handles 'unknown' and variations
    if pd.isna(a) or 'unk' in a.lower():
        return pd.NA
    return remove_apartment_info(a.lower())


In [ ]:
df2['cleaned_address'] = df2.apply(lambda row: clean_address(row), axis=1)
df2['cleaned_address'].value_counts(dropna=False)

In [ ]:
df3 = df2[df2['cleaned_address'].notna()]
df3.shape

In [ ]:
f"We removed {df.shape[0] - df3.shape[0]} records due to 'null-like' addresses"

See the benefit of cleaned addresses below:

In [ ]:
df3[['Incident Address','cleaned_address']].sample(2)

In [ ]:
df3.columns

In [ ]:
def make_address(row) -> str:
    street = row['cleaned_address'].strip()
    res_city_valid = pd.notna(row['Residence City'])
    inc_city_valid = pd.notna(row['Incident City'])
    if pd.notna(row['Incident City']):
        city = row['Incident City'].title().strip()
    elif pd.isna(row['Incident City']) and pd.notna(row['Residence City']):
        city = row['Residence City'].title().strip()
    else:
        city = ''
    zip_code = '' if pd.isna(row['Incident Zip Code']) else row['Incident Zip Code'].strip()
    address = f"{street} {city} {zip_code}"
    return address.strip()


def geo_query(row):
    address = make_address(row)
    url = f"https://my-geocoder.herokuapp.com/geocode?address={requests.utils.quote(address)}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise ValueError("Invalid response")


def self_geocode(row) -> tuple[float, float]:
    address = make_address(row)
    geocoder = ArcGIS()
    coded = geocoder.geocode(address)
    if coded:
        return {
            "latitude": coded.latitude,
            "longitude": coded.longitude,
            "score": coded.raw.get('score')
        }
    else:
        return None

In [ ]:
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm
tqdm.pandas()

df3['full_address'] = df3.apply(lambda row: make_address(row), axis=1)
df3['full_address'].head()

In [ ]:
geolocator = ArcGIS()
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0)
df3['geo_location'] = df3['full_address'].progress_apply(geocode)

In [ ]:
df3['coded_lat'] = df3['geo_location'].apply(lambda x: x.latitude if pd.notna(x) else None)
df3['coded_long'] = df3['geo_location'].apply(lambda x: x.longitude if pd.notna(x) else None)
df3['coded_score'] = df3['geo_location'].apply(lambda x: x.raw.get('score') if pd.notna(x) else None)

df3.drop('geo_location', axis=1, inplace=True)
print(df3.coded_score.describe())

## Calculate geopy distance from original lat/long

In [ ]:
from geopy import distance

def calc_distance(row):
    if pd.isna(row.latitude) or pd.isna(row.longitude) or pd.isna(row.coded_lat) or pd.isna(row.coded_long):
        return None
    d = distance.distance(
        (row.latitude, row.longitude),
        (row.coded_lat, row.coded_long)
    ).km
    return d

In [ ]:
df3['distance'] = df3.apply(lambda row: calc_distance(row), axis=1)

In [ ]:
df3.distance.describe().round(2)

In [ ]:
df3.coded_score.describe()

In [ ]:
df3.to_csv('../data/version3.csv', index=False)